In [140]:
from PIL import Image, ImageFilter, ImageOps
from IPython.display import display
import numpy as np
import requests
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

colors = [
    (0, 0, 0),
    (102, 102, 102),
    (0, 23, 246),
    (255, 255, 255),
    (170, 170, 170),
    (38, 201, 255),
    (0, 141, 38),
    (169, 35, 12),
    (150, 65, 18),
    (0, 255, 77),
    (255, 0, 19),
    (255, 120, 41),
    (176, 112, 28),
    (153, 0, 78),
    (147, 104, 103),
    (255, 201, 38),
    (255, 0, 143),
    (254, 175, 168),
    (0, 217, 163),
    (133, 178, 0),
    (128, 0, 255),
    (5, 44, 108),
    (185, 115, 255),
    (255, 247, 63),
]


def color_diff(rgb1, rgb2):
    color1_rgb = sRGBColor(*rgb1, is_upscaled=True)
    color2_rgb = sRGBColor(*rgb2, is_upscaled=True)
    color1_lab = convert_color(color1_rgb, LabColor)
    color2_lab = convert_color(color2_rgb, LabColor)
    delta_e = delta_e_cie2000(color1_lab, color2_lab)
    return delta_e


def resize_based_on_width(im, basewidth=50):
    wpercent = basewidth / float(im.size[0])
    hsize = int((float(im.size[1]) * float(wpercent)))
    im = im.resize((basewidth, hsize), Image.Resampling.NEAREST)
    return im


def preprocess(im):
    print("原圖：")
    display(im)
    print("縮放後：")
    im = resize_based_on_width(im)
    display(im)
    print("色彩量化：")
    im = im.quantize(18)
    display(im)
    im = im.convert("RGB")
    a = np.asarray(im)
    return a


color_id_map = {}


def make_line(a):
    line = []
    head = None
    for i in range(len(a) - 1):
        if i == 0:
            head = i
        else:
            if (a[i] != a[i + 1]).all():
                tail = i

                min_color_id = None
                if tuple(a[i]) not in color_id_map:
                    for j in range(len(colors)):
                        delta = color_diff(a[i], colors[j])
                        if min_color_id is None:
                            min_color_id = j
                            min_delta = delta
                        else:
                            if delta < min_delta:
                                min_color_id = j
                                min_delta = delta
                    color_id_map[tuple(a[i])] = min_color_id
                else:
                    min_color_id = color_id_map[tuple(a[i])]

                line += [(head, tail, min_color_id)]
                # next
                head = i + 1
    return line


def make_lines(a):
    lines = []
    for a_ in a:
        lines += [make_line(a_)]
    return lines


# im = Image.open("test4.jpg")
# # print(preprocess(im))
# lines = make_lines(preprocess(im))
# print(lines)


In [141]:
from selenium.webdriver.common.action_chains import ActionChains

def draw(driver, elem, lines, xoffset=10, yoffset=10, gap=1, line_height=None):
    if line_height is None:
        line_height = gap

    action = ActionChains(driver)
    action.w3c_actions.pointer_action._duration = 1
    for i, line in enumerate(lines):
        # print(line)
        for seg in line:
            action.click(driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[2]/div[1]/div/div/div[1]/div[2]/div/div[2]/div[1]/div[{seg[2] + 1}]'))
            action.move_to_element_with_offset(
                elem, xoffset + seg[0] * gap, yoffset + i * line_height
            )
            if seg[0] == seg[1]:
                action.click()
            else:
                action.click_and_hold()
                action.move_to_element_with_offset(
                    elem, xoffset + seg[1] * gap, yoffset + i * line_height
                )
                action.release()
        action.perform()

In [142]:
# 啟動 Chrome 必須先下載 Chrome 驅動器(https://sites.google.com/chromium.org/driver/)
# 將 Chrome 驅動器放置同路徑下，或設定於系統環境變數中的 Path
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from base64 import b64decode
from bs4 import BeautifulSoup
import os

options = Options()
options.add_extension("uBlock-Origin.crx")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get("https://gartic.io/")

print("Chrome Browser Invoked")


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Asus\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\Asus\AppData\Local\Temp\ipykernel_26456\1511601557.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Chrome Browser Invoked


In [151]:
# 找出題目
driver.switch_to.window(driver.window_handles[0])
soup = BeautifulSoup(driver.page_source, "html.parser")
titles = soup.find("div", class_="word")
keyword = titles.text
print("題目：" + keyword)
# 開啟新分頁(圖片搜尋頁面)
google_image_url = "https://images.google.com"
strScript = 'window.open("' + google_image_url + '");'
driver.execute_script(strScript)
driver.switch_to.window(driver.window_handles[1])
driver.find_element(By.CLASS_NAME, "gLFyf.gsfi").send_keys(keyword + "卡通", Keys.ENTER)


題目：植物


In [153]:
# 儲存圖片
for i in range(1,2):
    img = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[' + str(i) + ']/a[1]/div[1]/img')
    img.screenshot('image.jpg')
    print('儲存圖片(image.jpg)：')
    display(Image.open('image.jpg'))

C:\Users\Asus\AppData\Local\Temp\ipykernel_26456\440750292.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  img = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[' + str(i) + ']/a[1]/div[1]/img')


In [155]:
# 畫畫
driver.close()
driver.switch_to.window(driver.window_handles[0])

wait = WebDriverWait(driver, 999999, 0.5)
element = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "hint")))
wait = WebDriverWait(driver, 999999, 0.5)
elem = wait.until(EC.element_to_be_clickable((By.TAG_NAME, "canvas")))

with Image.open("image.jpg") as im:
    if elem:
        a = preprocess(im)
        lines = make_lines(a)
        draw(driver=driver, elem=elem, lines=lines, gap=3.5, xoffset=100, yoffset=100)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
Backtrace:
	Ordinal0 [0x00C9D953+2414931]
	Ordinal0 [0x00C2F5E1+1963489]
	Ordinal0 [0x00B1C580+836992]
	Ordinal0 [0x00B3E53D+976189]
	Ordinal0 [0x00B3F756+980822]
	GetHandleVerifier [0x00F0CC62+2510274]
	GetHandleVerifier [0x00EFF760+2455744]
	GetHandleVerifier [0x00D2EABA+551962]
	GetHandleVerifier [0x00D2D916+547446]
	Ordinal0 [0x00C35F3B+1990459]
	Ordinal0 [0x00C3A898+2009240]
	Ordinal0 [0x00C3A985+2009477]
	Ordinal0 [0x00C43AD1+2046673]
	BaseThreadInitThunk [0x76106739+25]
	RtlGetFullPathName_UEx [0x77D48FEF+1215]
	RtlGetFullPathName_UEx [0x77D48FBD+1165]
